In [1]:
# test different pre-trained models against the images used as "test" set during the training

#Importation des packages (il faut encore en supprimer quelques uns)
import numpy as np 
from keras.applications import  resnet50
from sklearn import metrics 
import os

import pandas as pd # Pour manipuler des DataFrames pandas

import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline

from keras.models import Sequential # Pour construire un réseau de neurones
from keras.layers import Dense, Conv2D # Pour instancier une couche dense
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Activation,GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.models import Model
import cv2
import keras
from numpy.random import seed
import tensorflow

from keras.preprocessing.image import ImageDataGenerator
import itertools # Pour créer des iterateurs

from sklearn import linear_model, preprocessing 



%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
from sklearn import metrics 

Using TensorFlow backend.


In [2]:
#Paramètres
base_img_paths="/home/marcpozzo/Desktop/c3po/Images_aquises/"
#generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_bigger.csv'
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur.csv'
#generateur_path='/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/Materiel/generateur.csv'
test_size=0.1
epochs=200
batch_size = 600
zoom_range = 1.25
horizontal_flip = True
Minimum_Number_Class=100
dropout_rate=0.1
#steps_per_epoch=len(data_train)//batch_size
steps_per_epoch=1
#validation_steps=len(data_test)//batch_size
validation_steps=1

#Importation du générateur contenant les chemins pour les imgettes grossies
#df=pd.read_csv("/home/marcpozzo/Desktop/c3po/Images_aquises/generateur.csv")
df=pd.read_csv(generateur_path)
df.drop('labels',inplace=True,axis=1)
df.shape


#Récupération des labels à partir du fichier
df["class"].unique()
for c in df:
    print(''+c+':',len(df[c].unique()))
df["class"].unique()

All_Unique=df["class"].unique()
Utilisable=[]
for i in df["class"].unique():
    if df["class"][df["class"]==i].count()>Minimum_Number_Class:
        Utilisable.append(i)
Utilisable
Non_Utilisable=set(All_Unique)-set(Utilisable)
Non_Utilisable
for i in Non_Utilisable:
    df=df[df["class"]!=i]
df=df[df["class"]!="oiseau"]  
df["class"].unique()


for i in range(len(df["class"])):
    image_name=df["img_paths"].iloc[i]
    df["img_paths"].iloc[i]=os.path.join(base_img_paths,image_name)
    
df.shape
df[df["img_paths"]=='/home/marcpozzo/Desktop/c3po/Images_aquises/Rec_images_bigger/EK000414_3_pigeon_178.JPG']

seed(1)
tensorflow.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 10,
        zoom_range = zoom_range,
        horizontal_flip = horizontal_flip
        )

test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = batch_size)


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = batch_size)


class: 12
img_paths: 6220
Found 5370 validated image filenames belonging to 6 classes.
Found 597 validated image filenames belonging to 6 classes.


In [3]:
#Conversion de l'image en array de format 28,28
def convert_image(X):
    X_img=[]
    for image in X:
        # Load image
        img=cv2.imread(image)
        # Resize image
        img=cv2.resize(img,(28,28))
        # for the black and white image
        if img.shape==(28, 28):
            img=img.reshape([28,28,1])
            img=np.concatenate([img,img,img],axis=2)
        # cv2 load the image BGR sequence color (not RGB)
        X_img.append(img[...,::-1])
    return np.array(X_img)

#Creation of dictionnary according to generator classes to convert the number labels (0,1,2,3,4,5) to tring labels ("autre","corbeau",....)
def metrique(arg_predict):
    dictionnaire=train_generator.class_indices
    dictionnaire_inv = {v: k for k, v in dictionnaire.items()}
    
    Keys=[]
    Values=[]

    for i in range(len(arg_predict)) :
        Keys.append(arg_predict[i])
        Values.append(dictionnaire_inv[arg_predict[i]])

    #print(metrics.classification_report(Y_test, Values))
    return Values

X_train_img = convert_image(data_train.img_paths)
Y_train = data_train['class']

# Load the images test
X_test_img = convert_image(data_test.img_paths)
Y_test = data_test['class']

In [6]:
#Testing all the neurone modes and display the principal metrics and crosstables.
##The best score is 6c_rob trained with original dataset. 
folder="zoom_models/"
folder="/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/6classes/"
Neurones=['6c_rob','drop_out.50','Recover1','Recover2']
Neurones=['drop_out.50','Recover2','Recover3','Recover4','Recover5','Recover6','Recover7']
for n in Neurones:
    model = load_model(folder+n)
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    X_test_features = intermediate_layer_model.predict(preprocess_input(X_test_img))
    arg_predict=X_test_features.argmax(axis=1)


    pred=metrique(arg_predict)
    print("")
    print("Mesures de qualité du modèle: ", n)
    print("")
    print(n,metrics.classification_report(Y_test,pred,digits=5))
    #print("")
    #print("table de cotingeance pour le modeèle: ", n)
    print("")
    #print(pd.crosstab(Y_test,np.array(pred)))


Mesures de qualité du modèle:  drop_out.50

drop_out.50               precision    recall  f1-score   support

       autre    0.92818   0.98246   0.95455       342
   chevreuil    1.00000   0.92857   0.96296        14
   corneille    0.89423   0.86916   0.88152       107
      faisan    0.85294   0.87879   0.86567        33
       lapin    0.68750   0.61111   0.64706        18
      pigeon    0.97059   0.79518   0.87417        83

    accuracy                        0.91792       597
   macro avg    0.88891   0.84421   0.86432       597
weighted avg    0.91826   0.91792   0.91630       597



Mesures de qualité du modèle:  Recover2

Recover2               precision    recall  f1-score   support

       autre    0.88889   0.98246   0.93333       342
   chevreuil    0.86667   0.92857   0.89655        14
   corneille    0.86792   0.85981   0.86385       107
      faisan    0.96154   0.75758   0.84746        33
       lapin    0.66667   0.44444   0.53333        18
      pigeon    1.00000

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Mesures de qualité du modèle:  Recover5

Recover5               precision    recall  f1-score   support

       autre    0.96188   0.95906   0.96047       342
   chevreuil    0.73684   1.00000   0.84848        14
   corneille    0.82500   0.92523   0.87225       107
      faisan    0.84848   0.84848   0.84848        33
       lapin    0.76471   0.72222   0.74286        18
      pigeon    0.95522   0.77108   0.85333        83

    accuracy                        0.91457       597
   macro avg    0.84869   0.87101   0.85431       597
weighted avg    0.91893   0.91457   0.91438       597



Mesures de qualité du modèle:  Recover6

Recover6               precision    recall  f1-score   support

       autre    0.92562   0.98246   0.95319       342
   chevreuil    0.85714   0.42857   0.57143        14
   corneille    0.84615   0.92523   0.88393       107
      faisan    0.87500   0.84848   0.86154        33
       lapin    0.66667   0.55556   0.60606        18
      pigeon    1.00000   0.7

In [7]:
#Testing all the neurone modes and display the principal metrics and crosstables.
##The best score is 6c_rob trained with original dataset. 

folde='iteration_models/'
Neurones=['iteration_no_transformation','z1.2_r_20','z1.2_r_20_drpt_0.1','z1.2_r_20_hflip_rotation_10']
for n in Neurones:
    model = load_model(folde+n)
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    X_test_features = intermediate_layer_model.predict(preprocess_input(X_test_img))
    arg_predict=X_test_features.argmax(axis=1)


    pred=metrique(arg_predict)
    print("")
    print("Mesures de qualité du modèle: ", n)
    print("")
    print(n,metrics.classification_report(Y_test,pred,digits=5))
    print("")
    print("table de cotingeance pour le modeèle: ", n)
    print("")
    print(pd.crosstab(Y_test,np.array(pred)))


Mesures de qualité du modèle:  iteration_no_transformation

iteration_no_transformation               precision    recall  f1-score   support

       autre    0.80097   0.96491   0.87533       342
   chevreuil    0.66667   0.42857   0.52174        14
   corneille    0.91566   0.71028   0.80000       107
      faisan    0.50000   0.54545   0.52174        33
       lapin    0.50000   0.11111   0.18182        18
      pigeon    0.84906   0.54217   0.66176        83

    accuracy                        0.79899       597
   macro avg    0.70539   0.55042   0.59373       597
weighted avg    0.79935   0.79899   0.78339       597


table de cotingeance pour le modeèle:  iteration_no_transformation

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        330          1          2       7      0       2
chevreuil      8          6          0       0      0       0
corneille     21          1         76       5     

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Mesures de qualité du modèle:  z1.2_r_20_hflip_rotation_10

z1.2_r_20_hflip_rotation_10               precision    recall  f1-score   support

       autre    0.81360   0.94444   0.87415       342
   chevreuil    0.72727   0.57143   0.64000        14
   corneille    0.69167   0.77570   0.73128       107
      faisan    0.66667   0.42424   0.51852        33
       lapin    0.00000   0.00000   0.00000        18
      pigeon    0.93478   0.51807   0.66667        83

    accuracy                        0.78894       597
   macro avg    0.63900   0.53898   0.57177       597
weighted avg    0.77392   0.78894   0.76819       597


table de cotingeance pour le modeèle:  z1.2_r_20_hflip_rotation_10

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        323          3         11       4      0       1
chevreuil      5          8          1       0      0       0
corneille     19          0         83       2     

In [12]:
#Testing all the neurone modes and display the principal metrics and crosstables.
##The best score is 6c_rob trained with original dataset. 
folder="2000epochs/"
Neurones=['all_transfo_pre_process','no_tranformation_gen_pre_process','avec_transfo_test','sans_transo','rot_5','decentrage_10','decentrage_10_bis','zoom_range_09_1.3','horizontal_flip']
for n in Neurones:
    model = load_model(folder+n)
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    X_test_features = intermediate_layer_model.predict(preprocess_input(X_test_img))
    arg_predict=X_test_features.argmax(axis=1)


    pred=metrique(arg_predict)
    print("")
    print("Mesures de qualité du modèle: ", n)
    print("")
    print(n,metrics.classification_report(Y_test,pred,digits=5))
    print("")
    print("table de cotingeance pour le modeèle: ", n)
    print("")
    print(pd.crosstab(Y_test,np.array(pred)))


Mesures de qualité du modèle:  all_transfo_pre_process

all_transfo_pre_process               precision    recall  f1-score   support

       autre    0.93352   0.98538   0.95875       342
   chevreuil    0.83333   0.35714   0.50000        14
   corneille    0.86486   0.89720   0.88073       107
      faisan    0.78788   0.78788   0.78788        33
       lapin    0.58333   0.38889   0.46667        18
      pigeon    0.97297   0.86747   0.91720        83

    accuracy                        0.90955       597
   macro avg    0.82932   0.71399   0.75187       597
weighted avg    0.90574   0.90955   0.90395       597


table de cotingeance pour le modeèle:  all_transfo_pre_process

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        337          0          1       3      0       1
chevreuil      3          5          5       0      1       0
corneille      5          0         96       2      3       1
f

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Mesures de qualité du modèle:  sans_transo

sans_transo               precision    recall  f1-score   support

       autre    0.84409   0.91813   0.87955       342
   chevreuil    0.71429   0.35714   0.47619        14
   corneille    0.77885   0.75701   0.76777       107
      faisan    0.46667   0.42424   0.44444        33
       lapin    0.33333   0.27778   0.30303        18
      pigeon    0.68116   0.56627   0.61842        83

    accuracy                        0.78057       597
   macro avg    0.63640   0.55009   0.58157       597
weighted avg    0.77044   0.78057   0.77232       597


table de cotingeance pour le modeèle:  sans_transo

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        314          2          7       5      1      13
chevreuil      8          5          0       0      1       0
corneille      9          0         81       7      5       5
faisan        12          0          

In [4]:
#Testing all the neurone modes and display the principal metrics and crosstables.
##The best score is 6c_rob trained with original dataset. 
folder="zoom_models/"
Neurones=['6c_rob','drop_out.50','z1.1','zoom_0.9:1.3_flip','zoom_1.3']
for n in Neurones:
    model = load_model(folder+n)
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    X_test_features = intermediate_layer_model.predict(preprocess_input(X_test_img))
    arg_predict=X_test_features.argmax(axis=1)


    pred=metrique(arg_predict)
    print("")
    print("Mesures de qualité du modèle: ", n)
    print("")
    print(n,metrics.classification_report(Y_test,pred,digits=5))
    print("")
    print("table de cotingeance pour le modeèle: ", n)
    print("")
    print(pd.crosstab(Y_test,np.array(pred)))


Mesures de qualité du modèle:  6c_rob

6c_rob               precision    recall  f1-score   support

       autre    0.97399   0.98538   0.97965       342
   chevreuil    0.93333   1.00000   0.96552        14
   corneille    0.94444   0.95327   0.94884       107
      faisan    0.91429   0.96970   0.94118        33
       lapin    0.73333   0.61111   0.66667        18
      pigeon    0.98718   0.92771   0.95652        83

    accuracy                        0.95980       597
   macro avg    0.91443   0.90786   0.90973       597
weighted avg    0.95902   0.95980   0.95902       597


table de cotingeance pour le modeèle:  6c_rob

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        337          1          0       1      3       0
chevreuil      0         14          0       0      0       0
corneille      3          0        102       1      1       0
faisan         1          0          0      32      